We want to obtain the requeriments of some games, as well as the description.

We'll scraping info from steam.

# Cyberpunk 2077

In [33]:
import requests 
from urllib.request import urlopen
from lxml import etree

In [34]:
# get html from site and write to local file
url = "https://store.steampowered.com/app/1091500/Cyberpunk_2077/"
headers = {'Content-Type': 'text/html',}
response = requests.get(url, headers=headers)
raw_html = response.text
html = etree.HTML(raw_html)


* Look For Description

In [35]:
xpath_for_description = "//div[(@id = 'game_area_description') and not(contains(text(), '\r')) ]/text()"


In [36]:
description_txt = html.xpath(xpath_for_description)

In [37]:
description_txt

[]

* Look for minimum requierments

In [38]:
xpath_for_min_req_strongs = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul//strong/text()"
xpath_for_min_req = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong

In [39]:
min_req_strongs = html.xpath(xpath_for_min_req_strongs)
min_req_txt = html.xpath(xpath_for_min_req)


In [40]:
min_req_strongs

['OS:',
 'Processor:',
 'Memory:',
 'Graphics:',
 'DirectX:',
 'Storage:',
 'Additional Notes:']

In [41]:
min_req_txt

[' Windows 7 or 10',
 ' Intel Core i5-3570K or AMD FX-8310',
 ' 8 GB RAM',
 ' NVIDIA GeForce GTX 780 or AMD Radeon RX 470',
 ' Version 12',
 ' 70 GB available space',
 ' In this game you will encounter a variety of visual effects that may provide seizures or loss of consciousness in a minority of people. If you or someone you know experiences any of the above symptoms while playing, stop and seek medical attention immediately.']

* Look for recommended  requierments

In [42]:
xpath_for_recommended_req_strongs = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul//strong/text()"
xpath_for_recommended_req = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong

In [43]:
recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
recommended_req_txt = html.xpath(xpath_for_recommended_req)

In [44]:
recommended_req_strongs

['OS:',
 'Processor:',
 'Memory:',
 'Graphics:',
 'DirectX:',
 'Storage:',
 'Additional Notes:']

In [45]:
recommended_req_txt

[' Windows 10',
 ' Intel Core i7-4790 or AMD Ryzen 3 3200G',
 ' 12 GB RAM',
 ' GTX 1060 6GB / GTX 1660 Super or Radeon RX 590',
 ' Version 12',
 ' 70 GB available space',
 ' SSD recommended']

In [46]:
merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]

In [47]:
merged_list_min

[('OS:', ' Windows 7 or 10'),
 ('Processor:', ' Intel Core i5-3570K or AMD FX-8310'),
 ('Memory:', ' 8 GB RAM'),
 ('Graphics:', ' NVIDIA GeForce GTX 780 or AMD Radeon RX 470'),
 ('DirectX:', ' Version 12'),
 ('Storage:', ' 70 GB available space'),
 ('Additional Notes:',
  ' In this game you will encounter a variety of visual effects that may provide seizures or loss of consciousness in a minority of people. If you or someone you know experiences any of the above symptoms while playing, stop and seek medical attention immediately.')]

In [48]:
import numpy as np 
merged_arr_min =np.array(merged_list_min)

In [49]:
merged_arr_min.shape

(7, 2)

In [50]:
merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs))  ]
merged_arr_recommended = np.array(merged_list_rec)

In [51]:
merged_arr_recommended

array([['OS:', ' Windows 10'],
       ['Processor:', ' Intel Core i7-4790 or AMD Ryzen 3 3200G'],
       ['Memory:', ' 12 GB RAM'],
       ['Graphics:', ' GTX 1060 6GB / GTX 1660 Super or Radeon RX 590'],
       ['DirectX:', ' Version 12'],
       ['Storage:', ' 70 GB available space'],
       ['Additional Notes:', ' SSD recommended']], dtype='<U47')

In [52]:
!pip install selenium

In [53]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (87.0.4280.66-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [65]:
import sys

In [66]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [222]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
#from webdriver_manager.firefox import GeckoDriverManager

In [68]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [227]:
def get_page_link_from_steam(game):
  print(f"Getting page for {game} ..."  )
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get("https://store.steampowered.com/")
  assert "Steam" in driver.title #assert is for debugging, if the condition is false, AssertionError is raised. 
  #Here we are using it to confirm that the title has "Steam".
  elem = driver.find_element_by_id("store_nav_search_term")
  elem.clear()
  elem.send_keys(game)
  elem.send_keys(Keys.ENTER)

 # print(driver.page_source)
 
  driver.find_element_by_xpath("//div[(@id = 'search_resultsRows')]/a[1]").click()
  source_str =  str(driver.page_source)
   #If not appropiate for all ages
  #print(source_str)
  if "View Page" in source_str:
    print("entered")

    #Specify the age
    element = driver.find_element_by_xpath("//select[@name='ageYear']")
    all_options = element.find_elements_by_tag_name("option")
    for option in all_options:
      #print("Value is: %s" % option.get_attribute("value"))
      if  option.get_attribute("value") == '1900':
        print("clicked")
        option.click()
        

    driver.find_element_by_xpath("//div[(@class = 'main_content_ctn')]/div[@class = 'agegate_text_container btns']/a[1]").click() 
    time.sleep(1)
  return driver.current_url
  


* Once we have the link that we need, let's scrap the page as we did it before.


In [108]:
def scrap_page_steam(link_path):
  print(f"Scraping {link_path} ..."  )
  headers = {'Content-Type': 'text/html',}
  response = requests.get(link_path, headers=headers)
  raw_html = response.text
  html = etree.HTML(raw_html)

#Get description
  xpath_for_description = "//div[(@id = 'game_area_description')]/text()"
  description_txt = html.xpath(xpath_for_description)

#Get Minimum Requirements
  xpath_for_min_req_strongs = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul//strong/text()"
  xpath_for_min_req = "//div[(@class = 'game_area_sys_req_leftCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong
  min_req_strongs = html.xpath(xpath_for_min_req_strongs)
  min_req_txt = html.xpath(xpath_for_min_req)

#Get Recommended Requirements
  xpath_for_recommended_req_strongs = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul//strong/text()"
  xpath_for_recommended_req = "//div[(@class='game_area_sys_req_rightCol')]/ul/ul/li[strong]/text()" #With [] we say to look for those li with strong
  recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
  recommended_req_txt = html.xpath(xpath_for_recommended_req)

#Merge strongs and text in the same array
  merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]
  merged_arr_min =np.array(merged_list_min)

  merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs)) ]
  merged_arr_recommended = np.array(merged_list_rec)

  return description_txt, merged_arr_min, merged_arr_recommended

In [229]:
page_to_scrap = get_page_link_from_steam('hellblade')
page_to_scrap


Getting page for hellblade ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


entered
clicked


'https://store.steampowered.com/app/414340/Hellblade_Senuas_Sacrifice/'

In [216]:
page_to_scrap

'https://store.steampowered.com/agecheck/app/1091500/#'

In [110]:
descr, min, recommended = scrap_page_steam(page_to_scrap)

Scraping https://store.steampowered.com/app/582010/Monster_Hunter_World/ ...


In [111]:
descr

['\r\n\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\tWelcome to a new world! Take on the role of a hunter and slay ferocious monsters in a living, breathing ecosystem where you can use the landscape and its diverse inhabitants to get the upper hand. Hunt alone or in co-op with up to three other players, and use materials collected from fallen foes to craft new gear and take on even bigger, badder beasts!',
 "As a hunter, you'll take on quests to hunt monsters in a variety of habitats.",
 'Take down these monsters and receive materials that you can use to create stronger weapons and armor in order to hunt even more dangerous monsters.',
 'In Monster Hunter: World, the latest installment in the series, you can enjoy the ultimate hunting experience, using everything at your disposal to hunt monsters in a new world teeming with surprises and excitement.',
 'Once every decade, elder dragons trek across the sea to travel to the land known as the New World in a migration referred to as the Elder Crossi

In [105]:
min

array([['OS:', ' WINDOWS® 7, 8, 8.1, 10 (64-bit required)'],
       ['Processor:', ' Intel® Core™ i5-4460, 3.20GHz or AMD FX™-6300'],
       ['Memory:', ' 8 GB RAM'],
       ['Graphics:',
        ' NVIDIA® GeForce® GTX 760 or AMD Radeon™ R7 260x (VRAM 2GB)'],
       ['DirectX:', ' Version 11'],
       ['Network:', ' Broadband Internet connection'],
       ['Storage:', ' 48 GB available space'],
       ['Sound Card:', ' DirectSound (DirectX® 9.0c)'],
       ['Additional Notes:',
        ' 1080p/30fps when graphics settings are set to "Low"']],
      dtype='<U59')

In [106]:
recommended

array([['OS:', ' WINDOWS® 7, 8, 8.1, 10 (64-bit required)'],
       ['Processor:',
        ' Intel® Core™ i7 3770 3.4GHz or Intel® Core™ i3 8350 4GHz or AMD Ryzen™ 5 1500X'],
       ['Memory:', ' 8 GB RAM'],
       ['Graphics:',
        ' NVIDIA® GeForce® GTX 1060 (VRAM 3GB) or AMD Radeon™ RX 570  (VRAM 4GB)'],
       ['DirectX:', ' Version 11'],
       ['Network:', ' Broadband Internet connection'],
       ['Storage:', ' 48 GB available space'],
       ['Sound Card:', ' DirectSound (DirectX® 9.0c or better)'],
       ['Additional Notes:',
        ' 1080p/30fps when graphics settings are set to "High"']],
      dtype='<U79')

* Let's scrap pcgamebenchmark.com

In [121]:
def get_game_link_pcgbm(game):
  print(f"Getting page for {game} ..."  )
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get("https://www.pcgamebenchmark.com/")
  assert "PCGameBenchmark" in driver.title #assert is for debugging, if the condition is false, AssertionError is raised. 
  #Here we are using it to confirm that the title has "Steam".
  elem = driver.find_element_by_xpath("//form[(@id = 'search')]/span/input[@name = 'q']")
  elem.clear()
  elem.send_keys(game)
  elem.send_keys(Keys.ENTER)
 
  #driver.find_element_by_xpath("//div[(@id = 'search_resultsRows')]/a[1]").click()
  #source_str =  str(driver.page_source)
  print(driver.current_url)

  
  return driver.current_url
  


In [173]:
def scrap_from_pcgbm(game_url):
  print(f"Scraping {game_url} ..."  )
  headers = {'Content-Type': 'text/html',}
  response = requests.get(game_url, headers=headers)
  raw_html = response.text
  html = etree.HTML(raw_html)

#Get description
  xpath_for_description = "//div[(@id = 'details-meta')]/div[@class = 'description']/p/text()"
  description_txt = html.xpath(xpath_for_description)


#Get Minimum Requirements
  xpath_for_min_req_strongs = "//div[(@class = 'row requirements')]/div[1]/ul/li/strong/text()"
  xpath_for_min_req = "//div[(@class = 'row requirements')]/div[1]/ul/li[strong]/text()" #With [] we say to look for those li with strong
  min_req_strongs = html.xpath(xpath_for_min_req_strongs)
  min_req_txt = html.xpath(xpath_for_min_req)

#Get Recommended Requirements
  xpath_for_recommended_req_strongs = "//div[(@class = 'row requirements')]/div[2]/ul/li/strong/text()"
  xpath_for_recommended_req = "//div[(@class = 'row requirements')]/div[2]/ul/li[strong]/text()"#With [] we say to look for those li with strong
  recommended_req_strongs = html.xpath(xpath_for_recommended_req_strongs)
  recommended_req_txt = html.xpath(xpath_for_recommended_req)

#Merge strongs and text in the same array
  merged_list_min = [(min_req_strongs[i], min_req_txt[i]) for i in range(0, len(min_req_strongs))]
  merged_arr_min =np.array(merged_list_min)

  merged_list_rec = [(recommended_req_strongs[i], recommended_req_txt[i]) for i in range(0, len(recommended_req_strongs)) ]
  merged_arr_recommended = np.array(merged_list_rec)


  return description_txt, merged_arr_min, merged_arr_recommended

In [196]:
pcgbm_link = get_game_link_pcgbm('valorant')

Getting page for valorant ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


https://www.pcgamebenchmark.com/valorant-system-requirements


In [190]:
description_txt_pcgbm, min_req_pcgbm, recommended_req_pcgbm = scrap_from_pcgbm(pcgbm_link)

Scraping https://www.pcgamebenchmark.com/grand-theft-auto-v-system-requirements ...


In [191]:
description_txt_pcgbm

['Grand Theft Auto 5 for PC offers players the option to explore the award-winning world of Los Santos and Blaine County in resolutions of up to 4k and beyond, as well as the chance to experience the game running at 60 frames per second.']

In [192]:
min_req_pcgbm

array([['Memory:', '4 GB'],
       ['Graphics Card:', 'NVIDIA GeForce 9800 GT'],
       ['CPU:', 'AMD Phenom 9850'],
       ['GTA 5 File Size:', '72 GB'],
       ['OS:',
        'Windows 10 64 Bit, Windows 8.1 64 Bit, Windows 8 64 Bit, Windows 7 64 Bit Service Pack 1']],
      dtype='<U88')

In [193]:
recommended_req_pcgbm

array([['Memory:', '8 GB'],
       ['Graphics Card:', 'NVIDIA GeForce GTX 660'],
       ['GTA 5 CPU:', 'Intel Core i5-3470'],
       ['File Size:', '72 GB'],
       ['OS:',
        'Windows 10 64 Bit, Windows 8.1 64 Bit, Windows 8 64 Bit, Windows 7 64 Bit Service Pack 1']],
      dtype='<U88')